Dowloading and Isntalling dependencies of spark in the next three coding blocks

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

Doing imports required

In [ ]:
!pip install -U imbalanced-learn
!pip install seaborn
from pyspark.sql.functions import col, expr, when
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_theme(style="whitegrid")
import matplotlib.pyplot as plt 
import matplotlib as mpl
import matplotlib.dates as mdates

Reading the contents of the json file, Please paste your link/address here.

In [ ]:
# Please paste the file link here.
DF = spark.read.json('/content/sample.json')

Identifying the Important Attributes for the class labeling.

In [ ]:
imp_Attributes = ['DownloadSpeed','UploadSpeed','Ping','TTFB']

Defining Function to calculate Median. Finding medians of required attributes.

In [ ]:
def calculate_Median(tdf,At):
  tdf = tdf.select(At)
  tdf = tdf.na.drop(subset=[At])
  count = tdf.count()
  tdf = tdf.sort(At)
  if count % 2 ==0:
    Median = (tdf.select(At).collect()[int(count/2)][0] + tdf.select(At).collect()[int(count/2)-1][0])/2
  else:
    Median = tdf.select(At).collect()[int((count-1)/2)][0]
  return Median

Med_DNS = calculate_Median(DF,imp_Attributes[0])
Med_UPS = calculate_Median(DF,imp_Attributes[1])
Med_Ping = calculate_Median(DF,imp_Attributes[2])
Med_TTFB = calculate_Median(DF,imp_Attributes[3])

In [ ]:
print (' Median DNS: ',Med_DNS,"\n",'Median UPS: ',Med_UPS,"\n",'Median Ping: ',Med_Ping,"\n",'Median TTFB: ',Med_TTFB )

All the Attributes to be used are defined here in this list.

In [ ]:
Attributes = ['Latitude','Longitude','@timestamp','Manufacturer','Model','AppName','SdkVersion','AppVersion',
              'AndroidApi','OSVersion','ActiveConnection','WifiBSSID','WifiRssi','WifiLinkSpeed',
              'lvl1_name','IP','ASN','ISP','URL','dnsLookup','initialConnection','sslNegotiation',
              'TTFB','fullyLoaded','PageloadSpeed','loadEventEnd','documentComplete','Ping',
              'DownloadSpeed','UploadSpeed','LteRssi','WcdmaRssi','GsmRssi']
# Attributes = ['location','@timestamp','Manufacturer','Model','AppName','SdkVersion','AppVersion',
#               'AndroidApi','OSVersion','ActiveConnection','WifiBSSID','WifiRssi','WifiLinkSpeed',
#               'lvl1_name','IP','ASN','ISP','URL','dnsLookup','initialConnection','sslNegotiation',
#               'TTFB','fullyLoaded','PageloadSpeed','loadEventEnd','documentComplete','Ping',
#               'DownloadSpeed','UploadSpeed','LteRssi','WcdmaRssi','GsmRssi']

In [ ]:
df = DF.select(Attributes)

Visualizing the Attributes in our data to make sure we are using required ones.

In [ ]:
df.columns

Removing Duplicates from the dataset samples.

In [ ]:
print("Checking and Removing Duplicates\n","Samples before removing:",df.count())
df = df.distinct()
print("Samples after removing Duplicates:",df.count())

Defining the Attributes according to the Action required for NULL values.

In [ ]:
Record_Removing_Attributes = ['Latitude','Longitude','@timestamp','AppName','SdkVersion','AppVersion','AndroidApi','OSVersion','ActiveConnection','lvl1_name','IP','ASN','ISP','URL','Manufacturer','Model']
Assign_Zero_Attributes_str = ['WifiBSSID','WifiRssi']
Assign_Zero_Attributes = ['WifiLinkSpeed','dnsLookup','initialConnection','sslNegotiation',
                         'fullyLoaded','PageloadSpeed','loadEventEnd','documentComplete','LteRssi','WcdmaRssi','GsmRssi']

Droping rows of NULL values against record removing attributes and with google URL

In [ ]:
df = df.na.drop(subset=Record_Removing_Attributes)
df = df[df.ActiveConnection != ""] # There were some values in this coulum that were empty but not assigned NULL position.
df=df.where(df.URL !="https://www.google.com.bh")
print("Truncated df count will be :", df.count())

In [ ]:
print('Before replacing values')
df

Replacement of NULL values with 0 and Medians accordingly

In [ ]:
df = df.fillna(value = Med_DNS ,subset = imp_Attributes[0])
df = df.fillna(value = Med_UPS ,subset = imp_Attributes[1])
df = df.fillna(value = Med_Ping ,subset = imp_Attributes[2])
df = df.fillna(value = Med_TTFB ,subset = imp_Attributes[3])
df = df.fillna(value=0,subset=Assign_Zero_Attributes)
df = df.fillna(value='0',subset=Assign_Zero_Attributes_str)
print('After replacing values')
df

Counting Acceptable values in data

In [ ]:
df.where((df.DownloadSpeed >= 1.5) & (df.UploadSpeed >= 0.5) & (df.Ping <=20) & (df.TTFB <=1200)).count()

Adding QoE Coulum

In [ ]:
df3 = df.withColumn("QoE", when(((df.DownloadSpeed >= 1.5) & (df.UploadSpeed >= 0.5) & (df.Ping <=20) & (df.TTFB <=1200)),1)
                                 .otherwise(0))

Understanding the storage type of data inside the coulums

In [ ]:
df3.printSchema()

Extracting name of coulums containing string data

In [ ]:
String_Cols = [item[0] for item in df3.dtypes if item[1].startswith('string')]

In [ ]:
print(String_Cols)

'gdf' contains frequency of each type of entry in string data frames.

In [ ]:
gdf = pd.DataFrame()
for x in String_Cols:
  t = df3.groupBy(x).count().toPandas()
  gdf = pd.concat([gdf,t],axis = 1)

Saving Frequency information of string coulums in "String_Frequency.csv" file.

In [ ]:
gdf.to_csv('/content/String_Frequency.csv')

Saving actual data after Cleansing to "After_Cleansing.csv" file

In [ ]:
pdf = df3.toPandas()

In [ ]:
pdf.to_csv('/content/After_Cleansing.csv')

It is saving the description of Data Distribution of all the coulums to "Distribution.csv" file 

In [ ]:
pdf.describe(include='all').to_csv('/content/Distribution.csv')

Code for using ADASYN technique of Oversampling for Data Balancing

In [ ]:
def makeOverSamplesADASYN(X,y):
 #input DataFrame
 #X →Independent Variable in DataFrame\
 #y →dependent Variable in Pandas DataFrame format
 from imblearn.over_sampling import ADASYN 
 sm = ADASYN()
 X, y = sm.fit_resample(X, y)
 return(X,y)

In [ ]:
P_df = df.select(imp_Attributes).toPandas()
Y_df = df3.select('QoE').toPandas()
P_df,Y_df = makeOverSamplesADASYN(P_df,Y_df)

In [ ]:
result = pd.concat([P_df,Y_df])

In [ ]:
result.to_csv('/content/After_ADASYN.csv')

Sample code for multi conditional work on spark

In [ ]:
# df2 = df.withColumn("QoE", when(((df.DownloadSpeed >= 1.5) & (df.DownloadSpeed < 5) & (df.UploadSpeed >= 0.5) & (df.UploadSpeed < 1.5) & (df.Ping >50)& (df.Ping <=100) & (df.TTFB >800)& (df.TTFB <=1200)) ,"A")
#                                  .when((df.DownloadSpeed >= 5) & (df.UploadSpeed >= 1.5) & (df.Ping <=50) & (df.TTFB <=800),"E")
#                                  .otherwise("NA"))

Code for removing Outliers

In [1]:
import numpy as np
from scipy import stats
df = df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]  # Will remove outliers 3 Standard Deviations from mean

Code for Normalization of values in the data frame

In [ ]:
# copy the data
df_min_max_scaled = df.copy()
  
# apply normalization techniques
for column in df_min_max_scaled.columns:
    df_min_max_scaled[column] = (df_min_max_scaled[column] - df_min_max_scaled[column].min()) / (df_min_max_scaled[column].max() - df_min_max_scaled[column].min())  